In [7]:
!git clone https://github.com/AlphaJugger11/mlopsquiz1.git

Cloning into 'mlopsquiz1'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hassankhaled21/egyptian-real-estate-listings")

print("Path to dataset files:", path)

100%|██████████| 7.08M/7.08M [00:01<00:00, 6.89MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/hassankhaled21/egyptian-real-estate-listings/versions/1


In [10]:
!cp -r /root/.cache/kagglehub/datasets/hassankhaled21/egyptian-real-estate-listings/versions/1 /content/drive/MyDrive/mlopsquiz1

In [14]:
import pandas as pd
import numpy as np
df = pd.read_csv("/content/drive/MyDrive/mlopsquiz1/1/egypt_real_estate_listings.csv")

In [15]:
df

,url,price,description,location,type,size,bedrooms,bathrooms,available_from,payment_method,down_payment
0,https://www.propertyfinder.eg/en/plp/buy/chale...,"8,000,000",OWN A CHALET IN EL GOUNA WITH A PRIME LOCATION...,"Swan Lake Gouna, Al Gouna, Hurghada, Red Sea",Chalet,732 sqft / 68 sqm,1+ Maid,1,31 Aug 2025,Cash,"1,200,000 EGP"
1,https://www.propertyfinder.eg/en/plp/buy/villa...,"25,000,000","For sale, a villa with immediate delivery in C...","Karmell, New Zayed City, Sheikh Zayed City, Giza",Villa,"2,368 sqft / 220 sqm",4,4,2 Sep 2025,Cash,"2,100,000 EGP"
2,https://www.propertyfinder.eg/en/plp/buy/chale...,"15,135,000","With a down payment of EGP 1,513,000, a fully ...","Azha North, Ras Al Hekma, North Coast",Chalet,"1,270 sqft / 118 sqm",2,2,19 Aug 2025,Cash,"1,513,000 EGP"
3,https://www.propertyfinder.eg/en/plp/buy/apart...,"12,652,000",Own an apartment in New Cairo with a minimal d...,"Taj City, 5th Settlement Compounds, The 5th Se...",Apartment,"1,787 sqft / 166 sqm",3,2,26 Aug 2025,Installments,"1,260,000 EGP"
4,https://www.propertyfinder.eg/en/plp/buy/villa...,"45,250,000",Project: Granville\nLocation: Fifth Settlement...,"Granville, New Capital City, Cairo",Villa,"4,306 sqft / 400 sqm",7,7,2 Sep 2025,Cash,"2,262,500 EGP"
...,...,...,...,...,...,...,...,...,...,...,...
19919,https://www.propertyfinder.eg/en/plp/buy/chale...,"10,500,000",Code: 2404\nChalet for sale in North Edge Towe...,"North Edge Towers, New Alamein City, Al Alamei...",Chalet,"1,259 sqft / 117 sqm",2,2,21 Aug 2025,Cash,NaN
19920,https://www.propertyfinder.eg/en/plp/buy/duple...,"24,000,000",Duplex for sale in Eastown -Sodic\n5th settlem...,"Eastown, 5th Settlement Compounds, The 5th Set...",Duplex,"2,756 sqft / 256 sqm",3+ Maid,4,1 Sep 2025,Cash,NaN
19921,https://www.propertyfinder.eg/en/plp/buy/chale...,"3,000,000",Chalet for Sale in One of the Prime Projects o...,"White Sand, Markaz Al Hamam, North Coast",Chalet,"1,195 sqft / 111 sqm",2,2,30 Jul 2025,Cash,NaN
19922,https://www.propertyfinder.eg/en/plp/buy/villa...,"9,500,000",Villa for sale in installments in Sarai Compou...,"Sarai, Mostakbal City Compounds, Mostakbal Cit...",Villa,"2,573 sqft / 239 sqm",5,4,23 Aug 2025,Cash,NaN


In [16]:
!cp  /content/mlopsquiz1/content/drive/MyDrive/mlopsquiz1/1/egypt_real_estate_listings.csv

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
print(train.shape)
print(test.shape)
# train.to_csv('train.csv', index=False)
# test.to_csv('test.csv', index=False)

(15939, 11)
(3985, 11)


In [22]:
# Handle missing values in the price column
df.dropna(subset=['price'], inplace=True)

# Extract the numerical value for size and convert to sqm
def convert_size_to_sqm(size_str):
    if isinstance(size_str, str):
        size_str = size_str.replace(',', '') # Handle commas
        parts = size_str.split('/')
        if len(parts) > 1:
            size_sqm = parts[1].strip().split()[0]
            try:
                return float(size_sqm)
            except ValueError:
                return np.nan
        elif 'sqft' in size_str:
            size_sqft = size_str.replace('sqft', '').strip()
            try:
                return float(size_sqft) / 10.764
            except ValueError:
                return np.nan
        else:
             try:
                return float(size_str.strip().split()[0])
             except ValueError:
                return np.nan
    return np.nan

df['size_sqm'] = df['size'].apply(convert_size_to_sqm)

# Extract the number of bedrooms and bathrooms
def extract_numerical_feature(feature_str):
    if isinstance(feature_str, str):
        if 'studio' in feature_str.lower():
            return 0
        elif '+' in feature_str:
            try:
                return int(feature_str.split('+')[0])
            except ValueError:
                return np.nan
        else:
            try:
                return int(feature_str)
            except ValueError:
                return np.nan
    return np.nan

df['bedrooms_num'] = df['bedrooms'].apply(extract_numerical_feature)
df['bathrooms_num'] = df['bathrooms'].apply(extract_numerical_feature)


# Create dummy variables for categorical columns
df = pd.get_dummies(df, columns=['type', 'location'])

# Drop the original categorical columns and the engineered size column
df.drop(['size', 'bedrooms', 'bathrooms', 'available_from', 'payment_method', 'down_payment'], axis=1, inplace=True)

# Separate features (X) and target (y)
X = df.drop('price', axis=1)
y = df['price']

display(X.head())
display(y.head())

,description,size_sqm,bedrooms_num,bathrooms_num,type_Apartment,type_Bulk Sale Unit,type_Bungalow,type_Cabin,type_Chalet,type_Duplex,...,"location_Zezenia, Hay Sharq, Alexandria","location_Zizinia Al Mostakbal, Mostakbal City Compounds, Mostakbal City - Future City, Cairo","location_Zizinia St., South Investors Area, New Cairo City, Cairo","location_Zoheira Abdeen St., El Yasmeen 5, El Yasmeen, New Cairo City, Cairo","location_Zomoroda, Qesm Borg El Arab, North Coast","location_Zomra East, The 5th Settlement, New Cairo City, Cairo","location_Zoya, Sidi Abdel Rahman, North Coast","location_il Bayou, Sahl Hasheesh, Hurghada, Red Sea","location_katameya coast, Qesm Ad Dabaah, North Coast","location_بوابة النعيم, Riviera City, Sheikh Zayed City, Giza"
0,OWN A CHALET IN EL GOUNA WITH A PRIME LOCATION...,68.0,1.0,1.0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,"For sale, a villa with immediate delivery in C...",220.0,4.0,4.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"With a down payment of EGP 1,513,000, a fully ...",118.0,2.0,2.0,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,Own an apartment in New Cairo with a minimal d...,166.0,3.0,2.0,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Project: Granville\nLocation: Fifth Settlement...,400.0,7.0,7.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


,price
0,8000000.0
1,25000000.0
2,15135000.0
3,12652000.0
4,45250000.0


In [25]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [26]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(X, y)

RandomForestRegressor()

In [27]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred_lr = model.predict(X)
mse_lr = mean_squared_error(y, y_pred_lr)
r2_lr = r2_score(y, y_pred_lr)

y_pred_rf = rf_model.predict(X)
mse_rf = mean_squared_error(y, y_pred_rf)
r2_rf = r2_score(y, y_pred_rf)

print(f"Linear Regression MSE: {mse_lr}")
print(f"Linear Regression R-squared: {r2_lr}")
print(f"Random Forest MSE: {mse_rf}")
print(f"Random Forest R-squared: {r2_rf}")

Linear Regression MSE: 222280849245846.0
Linear Regression R-squared: 0.6041738991366568
Random Forest MSE: 28322989959815.965
Random Forest R-squared: 0.9495639020697367


## Save models

### Subtask:
Save the trained models as pickle files.


**Reasoning**:
Save the trained Linear Regression and Random Forest models to pickle files.



In [28]:
import pickle

with open('linear_regression_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(rf_model, f)

## Copy model files

### Subtask:
Copy the saved model files to the specified directory.


**Reasoning**:
Copy the saved model files to the specified directory as instructed.



In [29]:
!cp linear_regression_model.pkl /content/mlopsquiz1/content/drive/MyDrive/mlopsquiz1/
!cp random_forest_model.pkl /content/mlopsquiz1/content/drive/MyDrive/mlopsquiz1/

cp: cannot create regular file '/content/mlopsquiz1/content/drive/MyDrive/mlopsquiz1/': No such file or directory
cp: cannot create regular file '/content/mlopsquiz1/content/drive/MyDrive/mlopsquiz1/': No such file or directory


**Reasoning**:
The previous copy command failed because the target directory structure seems incorrect. I will try copying to a slightly different path based on the previous successful copy command.



In [30]:
!cp linear_regression_model.pkl /content/drive/MyDrive/mlopsquiz1/
!cp random_forest_model.pkl /content/drive/MyDrive/mlopsquiz1/

In [31]:
!git add .


fatal: not a git repository (or any of the parent directories): .git
